In [ ]:
import pandas
import numpy
import scipy

from matplotlib.patches import Rectangle

from scipy.stats import shapiro
import scipy.stats as stats
from scipy.stats import chi2_contingency
import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn import decomposition
from sklearn.manifold import TSNE
from sklearn.manifold import MDS

import matplotlib.pyplot as plt 
import seaborn as sns
import matplotlib.patches as mpatches

from scipy.cluster.hierarchy import dendrogram, linkage

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
raw = pandas.read_table(pref_file, index_col=0)

In [ ]:
raw.head()

,LIVER,DARK_CHOCOLATE,MILK_CHOCOLATE,COFFEE,ORANGE_JUICE,LEMONS,SWEET_BREAD,RED_WINE,WHITE_WINE,BEER,...,BARLEY,BUTTER,WATERMELON,BLUEBERRY,BLACKBERRY,BANANA,RICE,BEET,WHIPPED_CREAM,MELON
id,,,,,,,,,,,,,,,,,,,,,
590597,3.0,5.0,2.0,2.0,5.0,4.0,4.0,NaN,NaN,NaN,...,3.0,4.0,5.0,4.0,NaN,5.0,3.0,5.0,NaN,2.0
591285,3.0,3.0,5.0,4.0,5.0,5.0,5.0,NaN,NaN,NaN,...,NaN,5.0,5.0,NaN,NaN,5.0,5.0,2.0,2.0,5.0
591286,2.0,1.0,5.0,5.0,4.0,3.0,5.0,2.0,1.0,3.0,...,NaN,5.0,5.0,2.0,2.0,5.0,5.0,2.0,3.0,5.0
591287,5.0,4.0,4.0,5.0,5.0,5.0,3.0,3.0,4.0,4.0,...,4.0,4.0,5.0,2.0,3.0,5.0,5.0,5.0,3.0,4.0
591288,4.0,3.0,5.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,...,3.0,5.0,5.0,NaN,1.0,5.0,5.0,4.0,5.0,5.0


In [ ]:
# compute correlation matrix

corr_mat = raw.drop(['ARTICHOKES'], axis=1).corr()

# compute dendrogram and add cl to corre_mat  
Z = linkage(corr_mat, method='average', metric='euclidean')
cl = scipy.cluster.hierarchy.fcluster(Z, 6, criterion='maxclust')

corr_mat['cl'] = ['cluster_{}'.format(i) for i in cl]

In [ ]:
corr_mat_1 = raw.drop(['ARTICHOKES'], axis=1).corr().unstack()
#raw.corr().loc['BLACKBERRY'].sort_values()

In [ ]:
corr_mat_1[corr_mat_1 < 1].sort_values()

VODKA           MILK_CHOCOLATE   -0.150277
MILK_CHOCOLATE  VODKA            -0.150277
ORANGE_JUICE    VODKA            -0.143312
VODKA           ORANGE_JUICE     -0.143312
                CREAM_BISCUITS   -0.129757
                                    ...   
WHITE_WINE      RED_WINE          0.671276
MUTTON          LAMB              0.675250
LAMB            MUTTON            0.675250
BLACKBERRY      BLUEBERRY         0.794162
BLUEBERRY       BLACKBERRY        0.794162
Length: 6006, dtype: float64

In [ ]:
network_pal = sns.color_palette("tab10")[0:len(corr_mat.cl.sort_values().unique())]
network_lut = dict(zip(corr_mat.cl.sort_values().unique(), network_pal))
networks = corr_mat.cl
network_colors = pandas.Series(networks).map(network_lut)

In [ ]:
sns.set(font_scale=0.41)
g = sns.clustermap(corr_mat.drop('cl', axis=1), center=0, vmin=-1, vmax=1, cmap="vlag", linewidths=0.001, linecolor='grey', row_colors=network_colors, col_colors=network_colors)

ax = g.ax_heatmap
ax.add_patch(Rectangle((0, 0), 5, 5, fill=False, edgecolor='black', lw=1))
ax.add_patch(Rectangle((5, 5), 3, 3, fill=False, edgecolor='black', lw=1))
ax.add_patch(Rectangle((8, 8), 8, 8, fill=False, edgecolor='black', lw=1))
ax.add_patch(Rectangle((16, 16), 22, 22, fill=False, edgecolor='black', lw=1))
ax.add_patch(Rectangle((38, 38), 19, 19, fill=False, edgecolor='black', lw=1))
ax.add_patch(Rectangle((57, 57), 22, 22, fill=False, edgecolor='black', lw=1))

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

for cl in range(1,7):
    cl_food = corr_mat.loc[corr_mat['cl'] == 'cluster_' + str(cl)].index
    cl_food.to_series().to_csv('data/cluster_' + str(cl) + '_foods_final.tsv', index=False, header=False)
    print(cl, cronbach_alpha(corr_mat.loc[cl_food,cl_food]))

1 0.796760189761048
2 0.6870239187781998
3 0.7729249701246628
4 0.8875929004294759
5 0.861346108352091
6 0.8752215640701054
